In [ ]:
# education_train_text_segmentation
import codecs
import os
 
file1 = open('drive/' + 'MyDrive/' + 'Corpus/' + 'corpus_education/' + 'education_en' + '.txt','w')
file2 = open('drive/' + 'MyDrive/' + 'Corpus/' + 'corpus_education/' + 'education_cn' + '.txt','w')
def separate_data(fpath):
    content = codecs.open(fpath, 'r', 'utf-8').readlines()
    for x in range(len(content[0:200000])):
        if x % 2 == 0:
            file1.write(content[x])
        else:
            file2.write(content[x])
 
file_path = 'drive/MyDrive/Corpus/corpus_education/Bi-Education.txt'
text1 = separate_data(file_path)

In [ ]:
# education_test_text_segmentation
import codecs
import os
 
file3 = open('drive/' + 'MyDrive/' + 'Corpus/' + 'corpus_education/' + 'education_en_test' + '.txt','w')
file4 = open('drive/' + 'MyDrive/' + 'Corpus/' + 'corpus_education/' + 'education_cn_test' + '.txt','w')
def separate_data(fpath):
    content = codecs.open(fpath, 'r', 'utf-8').readlines()
    for x in range(len(content)):
        if x % 2 == 0:
            file3.write(content[x])
        else:
            file4.write(content[x])
 
file_path2 = 'drive/MyDrive/Corpus/corpus_education/education_test.txt'
text2 = separate_data(file_path2)

In [ ]:
# word segmentation
 
import jieba
# Make Chinese word segmentation of sentences
def seg_depart(sentence):
    # Chinese word segmentation for each line in the document
    # print("doing word segmentation")
    sentence_depart = jieba.cut(sentence.strip())
    outstr = ''
    for word in sentence_depart:
        if word != '\n':
            outstr += word
            outstr += " "
    return outstr
 
# Give the document path
filename_train = "drive/MyDrive/Corpus/corpus_education/education_cn.txt"
outfilename_train = "drive/MyDrive/Corpus/corpus_education/education_out.txt"
inputs_train = open(filename_train, 'r', encoding='UTF-8')
outputs_train = open(outfilename_train, 'w', encoding='UTF-8')
 
filename_test = "drive/MyDrive/Corpus/corpus_education/education_cn_test.txt"
outfilename_test = "drive/MyDrive/Corpus/corpus_education/education_out_test.txt"
inputs_test = open(filename_test, 'r', encoding='UTF-8')
outputs_test = open(outfilename_test, 'w', encoding='UTF-8')
 
# Write the output to out.txt
for line in inputs_test:
    line_seg = seg_depart(line)
    outputs_test.write(line_seg + '\n')
outputs_test.close()
inputs_test.close()
print("testing word segmentation successfully！！！")
 
for line in inputs_train:
    line_seg = seg_depart(line)
    outputs_train.write(line_seg + '\n')
outputs_train.close()
inputs_train.close()
print("training word segmentation successfully！！！")

testing word segmentation successfully！！！
training word segmentation successfully！！！


In [ ]:
# hyperparams
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import codecs
import regex
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu

# data
source_train = 'drive/MyDrive/Corpus/corpus_education/education_out.txt'
target_train = 'drive/MyDrive/Corpus/corpus_education/education_en.txt'
source_test = 'drive/MyDrive/Corpus/corpus_education/education_out_test.txt'
target_test = 'drive/MyDrive/Corpus/corpus_education/education_en_test.txt'

# training
batch_size = 32 # alias = N
lr = 0.0001 # learning rate.
logdir = 'drive/MyDrive/logdir_education' # log directory

# model
maxlen = 30 # Maximum number of words in a sentence. alias = T.
min_cnt = 20 # words whose occurred less than min_cnt are encoded as <UNK>.
hidden_units = 512 # alias = C
num_blocks = 6 # number of encoder/decoder blocks
num_epochs = 20 # traversal time
num_heads = 8
dropout_rate = 0.1
sinusoid = False # select embedding method

In [ ]:
# data preprocessing
import codecs
import os
import regex
from collections import Counter

def make_vocab(fpath, fname):
    text = codecs.open(fpath, 'r', 'utf-8').read()
    # text = regex.sub("<[^>]+>", "", text)
    words = text.split()
    word2cnt = Counter(words)
    if not os.path.exists('drive/MyDrive/Corpus/corpus_education/preprocessed'): os.mkdir('drive/MyDrive/Corpus/corpus_education/preprocessed')
    with codecs.open('drive/MyDrive/Corpus/corpus_education/preprocessed/{}'.format(fname), 'w', 'utf-8') as fout:
        fout.write("{}\t1000000000\n{}\t1000000000\n{}\t1000000000\n{}\t1000000000\n".format("<PAD>", "<UNK>", "<S>", "</S>"))
        for word, cnt in word2cnt.most_common(len(word2cnt)):
            fout.write(u"{}\t{}\n".format(word, cnt))

make_vocab(source_train, "cn.vocab.tsv")
make_vocab(target_train, "en.vocab.tsv")
print("Done")

Done


In [ ]:
def load_cn_vocab():
    vocab = [line.split()[0] for line in codecs.open('drive/MyDrive/Corpus/corpus_education/preprocessed/cn.vocab.tsv', 'r', 'utf-8').read().splitlines() if int(line.split()[1])>=min_cnt]
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for idx, word in enumerate(vocab)}
    return word2idx, idx2word

def load_en_vocab():
    vocab = [line.split()[0] for line in codecs.open('drive/MyDrive/Corpus/corpus_education/preprocessed/en.vocab.tsv', 'r', 'utf-8').read().splitlines() if int(line.split()[1])>=min_cnt]
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for idx, word in enumerate(vocab)}
    return word2idx, idx2word

def create_data(source_sents, target_sents): 
    cn2idx, idx2cn = load_cn_vocab()
    en2idx, idx2en = load_en_vocab()
    
    # Index
    x_list, y_list, Sources, Targets = [], [], [], []
    for source_sent, target_sent in zip(source_sents, target_sents):
        x = [cn2idx.get(word, 1) for word in (source_sent + u" </S>").split()] # 1: OOV, </S>: End of Text
        y = [en2idx.get(word, 1) for word in (target_sent + u" </S>").split()] 
        if max(len(x), len(y)) <=maxlen:
            x_list.append(np.array(x))
            y_list.append(np.array(y))
            Sources.append(source_sent)
            Targets.append(target_sent)
    
    # Pad      
    X = np.zeros([len(x_list), maxlen], np.int32)
    Y = np.zeros([len(y_list), maxlen], np.int32)
    for i, (x, y) in enumerate(zip(x_list, y_list)):
        X[i] = np.lib.pad(x, [0, maxlen-len(x)], 'constant', constant_values=(0, 0))
        Y[i] = np.lib.pad(y, [0, maxlen-len(y)], 'constant', constant_values=(0, 0))
    
    return X, Y, Sources, Targets

def load_train_data():
    de_sents = [regex.sub("<[^>]+>", "", line) for line in codecs.open(source_train, 'r', 'utf-8').read().split("\n") if line and line[0] != "<"]
    en_sents = [regex.sub("<[^>]+>", "", line) for line in codecs.open(target_train, 'r', 'utf-8').read().split("\n") if line and line[0] != "<"]
    
    X, Y, Sources, Targets = create_data(de_sents, en_sents)
    return X, Y
    
def load_test_data():
    def _refine(line):
        line = regex.sub("<[^>]+>", "", line)
        line = regex.sub("<[^>]+>", "", line)
        return line.strip()
    
    de_sents = [_refine(line) for line in codecs.open(source_test, 'r', 'utf-8').read().split("\n") if line ]
    en_sents = [_refine(line) for line in codecs.open(target_test, 'r', 'utf-8').read().split("\n") if line ]
        
    X, Y, Sources, Targets = create_data(de_sents, en_sents)
    return X, Sources, Targets # (1064, 150) 

def load_test_data1():
    def _refine(line):
        line = regex.sub("<[^>]+>", "", line)
        line = regex.sub("<[^>]+>", "", line)
        return line.strip()
    
    de_sents = [_refine(line) for line in codecs.open(source_test1, 'r', 'utf-8').read().split("\n") if line ]
    en_sents = [_refine(line) for line in codecs.open(target_test1, 'r', 'utf-8').read().split("\n") if line ]
        
    X, Y, Sources, Targets = create_data(de_sents, en_sents)
    return X, Sources, Targets # (1064, 150) 

def get_batch_data():
    X, Y = load_train_data()
    
    # calculate batch size
    num_batch = len(X) // batch_size
    
    X = tf.convert_to_tensor(X, tf.int32)
    Y = tf.convert_to_tensor(Y, tf.int32)
    
    input_queues = tf.compat.v1.train.slice_input_producer([X, Y])
            
    x, y = tf.compat.v1.train.shuffle_batch(input_queues,
                                num_threads=8,
                                batch_size=batch_size, 
                                capacity=batch_size*64,   
                                min_after_dequeue=batch_size*32, 
                                allow_smaller_final_batch=False)
    
    return x, y, num_batch # (N, T), (N, T), ()



In [ ]:
# layer normalization
def normalize(inputs, epsilon = 1e-8, scope="ln", reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]
    
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        beta= tf.Variable(tf.zeros(params_shape))
        gamma = tf.Variable(tf.ones(params_shape))
        normalized = (inputs - mean) / ( (variance + epsilon) ** (.5) )
        outputs = gamma * normalized + beta
        
    return outputs

# initial embedding operation
def embedding(inputs, vocab_size, num_units, zero_pad=True, scale=True,scope="embedding", reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        lookup_table = tf.get_variable('lookup_table',
                                       dtype=tf.float32,
                                       shape=[vocab_size, num_units],
                                       initializer=tf.keras.initializers.glorot_normal()) 
        if zero_pad:
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),
                                      lookup_table[1:, :]), 0)
        outputs = tf.nn.embedding_lookup(lookup_table, inputs) # complete word embedding, change into 3d tensor
        
        if scale:
            outputs = outputs * (num_units ** 0.5) # scale
            
    return outputs
    
# positional encoding
def positional_encoding(inputs, num_units,zero_pad=True, scale=True, scope="positional_encoding", reuse=None):

    N, T = inputs.get_shape().as_list()
    with tf.variable_scope(scope, reuse=reuse):
        
        #position indices
        position_ind = tf.tile(tf.expand_dims(tf.range(T), 0), [N, 1])

        position_enc = np.array([
            [pos / np.power(10000, 2.*i/num_units) for i in range(num_units)]
            for pos in range(T)])

        position_enc[:, 0::2] = np.sin(position_enc[:, 0::2])  
        position_enc[:, 1::2] = np.cos(position_enc[:, 1::2])  

        lookup_table = tf.convert_to_tensor(position_enc)

        if zero_pad:
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),
                                      lookup_table[1:, :]), 0)
        outputs = tf.nn.embedding_lookup(lookup_table, position_ind)

        if scale:
            outputs = outputs * num_units**0.5

        return outputs


# attention mechanism
def multihead_attention(queries, keys, num_units=None, num_heads=8, dropout_rate=0, is_training=True, causality=False, scope="multihead_attention", reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        # Set the fall back option for num_units
        if num_units is None:
            num_units = queries.get_shape().as_list[-1]
        
        # Linear projections
        Q = tf.layers.dense(queries, num_units, activation=tf.nn.relu) # (N, T_q, C)
        K = tf.layers.dense(keys, num_units, activation=tf.nn.relu) # (N, T_k, C)
        V = tf.layers.dense(keys, num_units, activation=tf.nn.relu) # (N, T_k, C)
        
        # Split and concat
        Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0) # (h*N, T_q, C/h) 
        K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 
        V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 

        # Multiplication
        outputs = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
        
        # Scale
        outputs = outputs / (K_.get_shape().as_list()[-1] ** 0.5)
        
        # Key Masking
        key_masks = tf.sign(tf.abs(tf.reduce_sum(keys, axis=-1))) # (N, T_k)
        key_masks = tf.tile(key_masks, [num_heads, 1]) # (h*N, T_k)
        key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, tf.shape(queries)[1], 1]) # (h*N, T_q, T_k)
        
        paddings = tf.ones_like(outputs)*(-2**32+1)
        outputs = tf.where(tf.equal(key_masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Causality = Future blinding
        if causality:
            diag_vals = tf.ones_like(outputs[0, :, :]) # (T_q, T_k)
            tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1]) # (h*N, T_q, T_k)
   
            paddings = tf.ones_like(masks)*(-2**32+1)
            outputs = tf.where(tf.equal(masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Activation
        outputs = tf.nn.softmax(outputs) # (h*N, T_q, T_k)
         
        # Query Masking
        query_masks = tf.sign(tf.abs(tf.reduce_sum(queries, axis=-1))) # (N, T_q)
        query_masks = tf.tile(query_masks, [num_heads, 1]) # (h*N, T_q)
        query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, tf.shape(keys)[1]]) # (h*N, T_q, T_k)
        outputs *= query_masks # broadcasting. (h*N, T_q, T_k)
          
        # Dropouts avoid overfitting
        outputs = tf.layers.dropout(outputs, rate=dropout_rate, training=tf.convert_to_tensor(is_training))
               
        # Weighted sum
        outputs = tf.matmul(outputs, V_) # ( h*N, T_q, C/h)
        
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2 ) # (N, T_q, C)
              
        # Residual connection
        outputs += queries
              
        # Normalize
        outputs = normalize(outputs) # (N, T_q, C)
 
    return outputs


def feedforward(inputs, num_units=[2048, 512],scope="multihead_attention", reuse=None):

    with tf.variable_scope(scope, reuse=reuse):
        # Inner layer
        params = {"inputs": inputs, "filters": num_units[0], "kernel_size": 1,
                  "activation": tf.nn.relu, "use_bias": True}
        outputs = tf.layers.conv1d(**params)
        
        # Readout layer
        params = {"inputs": outputs, "filters": num_units[1], "kernel_size": 1,
                  "activation": None, "use_bias": True}
        outputs = tf.layers.conv1d(**params)
        
        # Residual connection
        outputs += inputs
        
        # Normalize
        outputs = normalize(outputs)
    
    return outputs

# smoothing
def label_smoothing(inputs, epsilon=0.1):
    ''' https://arxiv.org/abs/1512.00567.'''

    K = inputs.get_shape().as_list()[-1] # number of channels
    return ((1-epsilon) * inputs) + (epsilon / K)

In [ ]:
class Graph():
    def __init__(self, is_training=True):
        self.graph = tf.Graph()
        with self.graph.as_default():
            if is_training:
                self.x, self.y, self.num_batch = get_batch_data() # (N, T)
            else: # inference
                self.x = tf.placeholder(tf.int32, shape=(None, maxlen))
                self.y = tf.placeholder(tf.int32, shape=(None, maxlen))

            # decoder input
            self.decoder_inputs = tf.concat((tf.ones_like(self.y[:, :1])*2, self.y[:, :-1]), -1) # 2:<S>

            # load   
            de2idx, idx2de = load_cn_vocab()
            en2idx, idx2en = load_en_vocab()
            
            # Encoder
            with tf.variable_scope("encoder"):
                ## embedding
                self.enc = embedding(self.x, 
                                      vocab_size=len(de2idx), 
                                      num_units=hidden_units, 
                                      scale=True,
                                      scope="enc_embed")
                
                ## positional embedding
                if sinusoid:
                    self.enc += positional_encoding(self.x,
                                      num_units=hidden_units, 
                                      zero_pad=False, 
                                      scale=False,
                                      scope="enc_pe")
                else:
                    self.enc += embedding(tf.tile(tf.expand_dims(tf.range(tf.shape(self.x)[1]), 0), [tf.shape(self.x)[0], 1]),
                                      vocab_size=maxlen, 
                                      num_units=hidden_units, 
                                      zero_pad=False, 
                                      scale=False,
                                      scope="enc_pe")
                    
                 
                ## dropout layer
                self.enc = tf.layers.dropout(self.enc, 
                                            rate=dropout_rate, 
                                            training=tf.convert_to_tensor(is_training))
                
                ## number of encoders: num_blocks
                for i in range(num_blocks):
                    with tf.variable_scope("num_blocks_{}".format(i)):
                        ### Multihead Attention
                        self.enc = multihead_attention(queries=self.enc, 
                                                        keys=self.enc, 
                                                        num_units=hidden_units, 
                                                        num_heads=num_heads, 
                                                        dropout_rate=dropout_rate,
                                                        is_training=is_training,
                                                        causality=False)
                        
                        ### Feed Forward
                        self.enc = feedforward(self.enc, num_units=[4*hidden_units, hidden_units])
            
            # decoder
            with tf.variable_scope("decoder"):
                self.dec = embedding(self.decoder_inputs, 
                                      vocab_size=len(en2idx), 
                                      num_units=hidden_units,
                                      scale=True, 
                                      scope="dec_embed")
                
                if sinusoid:
                    self.dec += positional_encoding(self.decoder_inputs,
                                      vocab_size=maxlen, 
                                      num_units=hidden_units, 
                                      zero_pad=False, 
                                      scale=False,
                                      scope="dec_pe")
                else:
                    self.dec += embedding(tf.tile(tf.expand_dims(tf.range(tf.shape(self.decoder_inputs)[1]), 0), [tf.shape(self.decoder_inputs)[0], 1]),
                                      vocab_size=maxlen, 
                                      num_units=hidden_units, 
                                      zero_pad=False, 
                                      scale=False,
                                      scope="dec_pe")
                
                self.dec = tf.layers.dropout(self.dec, 
                                            rate=dropout_rate, 
                                            training=tf.convert_to_tensor(is_training))
                
                for i in range(num_blocks):
                    with tf.variable_scope("num_blocks_{}".format(i)):
                        ## Multihead Attention ( self-attention)
                        self.dec = multihead_attention(queries=self.dec, 
                                                        keys=self.dec, 
                                                        num_units=hidden_units, 
                                                        num_heads=num_heads, 
                                                        dropout_rate=dropout_rate,
                                                        is_training=is_training,
                                                        causality=True, 
                                                        scope="self_attention")
                        
                        ## Multihead Attention (vanilla attention)
                        self.dec = multihead_attention(queries=self.dec, 
                                                        keys=self.enc, 
                                                        num_units=hidden_units, 
                                                        num_heads=num_heads,
                                                        dropout_rate=dropout_rate,
                                                        is_training=is_training, 
                                                        causality=False,
                                                        scope="vanilla_attention")
                        
                        ## Feed Forward
                        self.dec = feedforward(self.dec, num_units=[4*hidden_units, hidden_units])
                
            # Final linear projection
            self.logits = tf.layers.dense(self.dec, len(en2idx)) #shape: [N,T,len(en2idx)]
            self.preds = tf.to_int32(tf.arg_max(self.logits, dimension=-1))
            self.istarget = tf.to_float(tf.not_equal(self.y, 0))
            self.acc = tf.reduce_sum(tf.to_float(tf.equal(self.preds, self.y))*self.istarget)/ (tf.reduce_sum(self.istarget))
            tf.summary.scalar('acc', self.acc)
            
            if is_training:  
                # smoothing
                self.y_smoothed = label_smoothing(tf.one_hot(self.y, depth=len(en2idx)))
                self.loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y_smoothed)
                self.mean_loss = tf.reduce_sum(self.loss*self.istarget) / (tf.reduce_sum(self.istarget))
               
                # graph
                self.global_step = tf.Variable(0, name='global_step', trainable=False)
                self.optimizer = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.9, beta2=0.98, epsilon=1e-8)
                self.train_op = self.optimizer.minimize(self.mean_loss, global_step=self.global_step)
                   
                # summary
                tf.summary.scalar('mean_loss', self.mean_loss)
                self.merged = tf.summary.merge_all()

if __name__ == '__main__':                    
    de2idx, idx2de = load_cn_vocab()
    en2idx, idx2en = load_en_vocab()
    
    g = Graph("train"); print("load successfully")
    
    sv = tf.train.Supervisor(graph=g.graph, 
                             logdir=logdir,
                             save_model_secs=0)
    with sv.managed_session() as sess:
        for epoch in range(1, num_epochs+1): 
            print('The%d' % (epoch)+'period')
            if sv.should_stop(): break
            for step in tqdm(range(g.num_batch), total=g.num_batch, ncols=70, leave=False, unit='b'):
                sess.run(g.train_op)
                
            gs = sess.run(g.global_step)   
            sv.saver.save(sess, logdir + '/model_epoch_%02d_gs_%d' % (epoch, gs))
    
    print("Done")    

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:268: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  warnings.warn('`tf.layers.dropout` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:202: UserWarning: `tf.layers.conv1d` is deprecated and will be removed 

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

load successfully
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.


  0%|                                         | 0/2096 [00:00<?, ?b/s]

The1period
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.


...

  0%|                                         | 0/2096 [00:00<?, ?b/s]

The20period


 24%|███████▎                       | 498/2096 [01:14<03:56,  6.77b/s]

INFO:tensorflow:Recording summary at step 40322.


 36%|███████████▎                   | 761/2096 [01:53<03:17,  6.75b/s]

In [ ]:
def eval(): 
    # load graph
    g = Graph(is_training=False)
    print("Graph loaded")
    
    # load data
    X, Sources, Targets = load_test_data()
    cn2idx, idx2cn = load_cn_vocab()
    en2idx, idx2en = load_en_vocab()
     
     
    # Start session 
    with g.graph.as_default():    
        sv = tf.train.Supervisor()
        with sv.managed_session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
            ## restore parameters
            sv.saver.restore(sess, tf.train.latest_checkpoint(logdir))
            print("Restored!")
              
            ## Get model name
            mname = open(logdir + '/checkpoint', 'r').read().split('"')[1] # model name
             
            ## Inference
            if not os.path.exists('drive/Shareddrives/ruochen.katherina@gmail.com/results_education'): os.mkdir('drive/Shareddrives/ruochen.katherina@gmail.com/results_education')
            with codecs.open("drive/Shareddrives/ruochen.katherina@gmail.com/results_education/" + mname, "w", "utf-8") as fout:
                list_of_refs, hypotheses = [], []
                for i in range(len(X) // batch_size):
                     
                    ### Get mini-batches
                    x = X[i*batch_size: (i+1)*batch_size]
                    sources = Sources[i*batch_size: (i+1)*batch_size]
                    targets = Targets[i*batch_size: (i+1)*batch_size]
                     
                    ### Autoregressive inference
                    preds = np.zeros((batch_size, maxlen), np.int32)
                    for j in range(maxlen):
                        _preds = sess.run(g.preds, {g.x: x, g.y: preds})
                        preds[:, j] = _preds[:, j]
                     
                    ### Write to file
                    for source, target, pred in zip(sources, targets, preds): # sentence-wise
                        got = " ".join(idx2en[idx] for idx in pred).split("</S>")[0].strip()
                        fout.write("- source: " + source +"\n")
                        fout.write("- expected: " + target + "\n")
                        fout.write("- got: " + got + "\n\n")
                        print("- source: " + source +"\n")
                        print("- expected: " + target + "\n")
                        print("- got: " + got + "\n\n")
                        fout.flush()
                          
                        #bleu score
                        ref = target.split()
                        hypothesis = got.split()
                        if len(ref) > 3 and len(hypothesis) > 3:
                            list_of_refs.append([ref])
                            hypotheses.append(hypothesis)
              
                ## Calculate bleu score
                print(list_of_refs)
                score = corpus_bleu(list_of_refs, hypotheses)
                fout.write("Bleu Score = " + str(100*score))
                print("Bleu Score = " + str(100*score))
                                          
if __name__ == '__main__':
    eval()
    print("Done")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:268: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  warnings.warn('`tf.layers.dropout` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:202: UserWarning: `tf.layers.conv1d` is deprecated and will be removed 

Graph loaded
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Restoring parameters from drive/MyDrive/logdir_education/model_epoch_19_gs_39824
Restored!
- source: 这位 老 教授 是 在 1949 年 回到 中国 的 ， 当时 大陆 刚 解放 。

- expected: The old prefessor came back to Chine in 1949 when the mainland had just been liberated.

- got: <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>


- source: 全 社会 都 应 关心 孩子 的 健康成长 。

- expected: The whole society should be concerned about the health and sound growth of children.

- got: <UNK> <UNK> <UNK> <UNK> and <UNK> are all full of <UNK>


- source: 他们 为 自己 的 祖国 献出 了 生命 ， 被誉为 英雄 。

- expected: They gave their lives for their country and were honoured as heroes.

- got: <UNK> <UNK> <UNK> <UNK> and <UNK> have all been <UNK> from <UNK> and <UNK>


- source: 为了 不 留下 指纹 ， 他们 一定 戴上 了 手套 。

- expected: They mu

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
